In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

import chess

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [2]:
tqdm.pandas()

/usr/local/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
players = ['Konevlad','Crecent','LongLive16Hayastan','may6enexttime','IWANNABEADOORED',
           'Ogrilla','dolar9','MilfSalvatore','cutemouse83','Benefactorr',
           'cjota95','alireza2003','vistagausta','GRUrussia','Drvitman',
           'NIndja64','KnezMihailova','Bestinblitz','AnonLondonKiller','black_knight22',
           'IVK88','Inventing_Invention','temp006','Arka50','Ernst_Gruenfeld',
           'Zaola420','GeorgMeier','SindarovGM','BenjaminBokTwitch','Hungry_Dragon',
           'avantage_ru','Chesstoday','GABUZYAN_CHESSMOOD','Tryhard00','TilChess',
           'muisback','MikeGScarn','Zhalmakhanov_R','L04d1ng','BabaRamdev',
           'opperwezen','Feokl1995','LucaBrazzi','Venost17','RebeccaHarris',
           'gmmoranda','YuQuesada','FakeBruceLee','swimmerchess','GrigorGrigorov']

elen = 8
cur_cols = [f'c{i}' for i in range(elen)]
player_cols = [f'p{i}' for i in range(elen)]
optimal_cols = [f'o{i}' for i in range(elen)]
numeric_cols = cur_cols + player_cols + optimal_cols
diff_cols = ['cur_pos'] + cur_cols + ['player_uci'] + player_cols + ['optimal_uci'] + optimal_cols

In [23]:
def get_player_df(file_path, names):
    df = pd.read_csv(file_path, names=names, index_col=False)
    df['color'] = df['cur_pos'].apply(lambda x: x.split()[1])
#     df = df.drop('cur_pos', axis=1)
#     df.loc[df['color'] == 'b',cols] = df*-1
#     df = df.drop('color', axis=1)
    df['player'] = os.path.basename(file_path).split('.')[0]
    return df

def get_all_df(players):
    ans = [get_player_df(f'diffs/{i}.csv', diff_cols) for i in players]
    ans = pd.concat(ans).reset_index(drop=True)
    return ans

def get_diff_df(all_df, cols1, cols2):
    diff_df = all_df[cols1] - all_df[cols2].values
#     diff_df['player'] = all_df['player']
    return diff_df

def get_dummies(all_df, col1, col2=None):
    if col2:
        return pd.get_dummies(all_df[col1], dtype=int) - pd.get_dummies(all_df[col2], dtype=int)
    return pd.get_dummies(all_df[col1], dtype=int)

In [24]:
def san_to_piece(san):
    if san[0]=='K':
        return 'king'
    if san[0]=='Q':
        return 'queen'
    if san[0]=='B':
        return 'bishop'
    if san[0]=='N':
        return 'knight'
    if san[0]=='R':
        return 'rook'
    return 'pawn'

def get_piece(row):
    board = chess.Board(row['cur_pos'])
    player_san = board.san(chess.Move.from_uci(row['player_uci']))
    optimal_san = board.san(chess.Move.from_uci(row['optimal_uci']))
    row['player_piece'] = san_to_piece(player_san)
    row['optimal_piece'] = san_to_piece(optimal_san)
    return row

In [7]:
all_df = get_all_df(players).dropna().reset_index(drop=True)
all_df = all_df.progress_apply(get_piece, axis=1)

In [25]:
cur_pos = all_df[cur_cols]
y = all_df['player']

player_df1 = get_diff_df(all_df, player_cols, cur_cols)
player_df2 = get_dummies(all_df,'player_piece')
player_df3 = pd.concat([player_df1, player_df2], axis=1)
player_df4 = pd.concat([cur_pos, player_df1], axis=1)
player_df5 = pd.concat([cur_pos, player_df2], axis=1)
player_df6 = pd.concat([cur_pos, player_df1, player_df2], axis=1)

optimal_df1 = get_diff_df(all_df, optimal_cols, cur_cols)
optimal_df2 = get_dummies(all_df,'optimal_piece')
optimal_df3 = pd.concat([optimal_df1, optimal_df2], axis=1)
optimal_df4 = pd.concat([cur_pos, optimal_df1], axis=1)
optimal_df5 = pd.concat([cur_pos, optimal_df2], axis=1)
optimal_df6 = pd.concat([cur_pos, optimal_df1, optimal_df2], axis=1)

bias_df1 = get_diff_df(all_df, player_cols, optimal_cols)
bias_df2 = get_dummies(all_df,'player_piece','optimal_piece')
bias_df3 = pd.concat([bias_df1, bias_df2], axis=1)
bias_df4 = pd.concat([cur_pos, bias_df1], axis=1)
bias_df5 = pd.concat([cur_pos, bias_df2], axis=1)
bias_df6 = pd.concat([cur_pos, bias_df1, bias_df2], axis=1)

In [27]:
X1 = player_df1
X2 = optimal_df1
X3 = bias_df1

scores1_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores1_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores1_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [28]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores1_1.mean():.3f} - Player Change')
print(f'{scores1_2.mean():.3f} - Optimal Change')
print(f'{scores1_3.mean():.3f} - Player Bias')

Baseline Accuracy: 0.020

Random Forest
0.030 - Player Change
0.030 - Optimal Change
0.023 - Player Bias


In [29]:
X1 = player_df2
X2 = optimal_df2
X3 = bias_df2

scores2_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores2_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores2_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [30]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores2_1.mean():.3f} - Player Change')
print(f'{scores2_2.mean():.3f} - Optimal Change')
print(f'{scores2_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.022 - Player Change
0.022 - Optimal Change
0.022 - Player Bias



In [31]:
X1 = player_df3
X2 = optimal_df3
X3 = bias_df3

scores3_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores3_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores3_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [32]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores3_1.mean():.3f} - Player Change')
print(f'{scores3_2.mean():.3f} - Optimal Change')
print(f'{scores3_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.032 - Player Change
0.031 - Optimal Change
0.024 - Player Bias



In [33]:
X1 = player_df4
X2 = optimal_df4
X3 = bias_df4

scores4_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores4_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores4_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [34]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores4_1.mean():.3f} - Player Change')
print(f'{scores4_2.mean():.3f} - Optimal Change')
print(f'{scores4_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.038 - Player Change
0.036 - Optimal Change
0.034 - Player Bias



In [35]:
X1 = player_df5
X2 = optimal_df5
X3 = bias_df5

scores5_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores5_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores5_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [36]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores5_1.mean():.3f} - Player Change')
print(f'{scores5_2.mean():.3f} - Optimal Change')
print(f'{scores5_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.034 - Player Change
0.034 - Optimal Change
0.034 - Player Bias



In [37]:
X1 = player_df6
X2 = optimal_df6
X3 = bias_df6

scores6_1 = cross_val_score(RandomForestClassifier(), X1, y)
scores6_2 = cross_val_score(RandomForestClassifier(), X2, y)
scores6_3 = cross_val_score(RandomForestClassifier(), X3, y)

In [38]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores6_1.mean():.3f} - Player Change')
print(f'{scores6_2.mean():.3f} - Optimal Change')
print(f'{scores6_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.037 - Player Change
0.036 - Optimal Change
0.034 - Player Bias



In [41]:
def get_kmeans_classes(df, y):
    means_df = pd.concat([df,y],axis=1).groupby('player').mean()
    kmeans = KMeans(n_clusters=2).fit(means_df)
    d = dict(zip(means_df.index, kmeans.labels_))
    return y.map(d)

In [42]:
# KMeans Moves Only
X1 = player_df1
X2 = optimal_df1
X3 = bias_df1

y1 = get_kmeans_classes(player_df1,y)
y2 = get_kmeans_classes(optimal_df1,y)
y3 = get_kmeans_classes(bias_df1,y)

scores1_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores1_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores1_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [43]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores1_1.mean():.3f} - Player Change')
print(f'{scores1_2.mean():.3f} - Optimal Change')
print(f'{scores1_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.616 - Player Change
0.603 - Optimal Change
0.540 - Player Bias



In [44]:
X1 = player_df2
X2 = optimal_df2
X3 = bias_df2

y1 = get_kmeans_classes(player_df2,y)
y2 = get_kmeans_classes(optimal_df2,y)
y3 = get_kmeans_classes(bias_df2,y)

scores2_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores2_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores2_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [45]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores2_1.mean():.3f} - Player Change')
print(f'{scores2_2.mean():.3f} - Optimal Change')
print(f'{scores2_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.522 - Player Change
0.807 - Optimal Change
0.675 - Player Bias



In [46]:
# KMeans Moves Only
X1 = player_df3
X2 = optimal_df3
X3 = bias_df3

y1 = get_kmeans_classes(player_df3,y)
y2 = get_kmeans_classes(optimal_df3,y)
y3 = get_kmeans_classes(bias_df3,y)

scores3_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores3_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores3_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [47]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores3_1.mean():.3f} - Player Change')
print(f'{scores3_2.mean():.3f} - Optimal Change')
print(f'{scores3_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.615 - Player Change
0.626 - Optimal Change
0.708 - Player Bias



In [48]:
# KMeans Moves Only
X1 = player_df4
X2 = optimal_df4
X3 = bias_df4

y1 = get_kmeans_classes(player_df4,y)
y2 = get_kmeans_classes(optimal_df4,y)
y3 = get_kmeans_classes(bias_df4,y)

scores4_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores4_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores4_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [49]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores4_1.mean():.3f} - Player Change')
print(f'{scores4_2.mean():.3f} - Optimal Change')
print(f'{scores4_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.598 - Player Change
0.602 - Optimal Change
0.584 - Player Bias



In [50]:
# KMeans Moves Only
X1 = player_df5
X2 = optimal_df5
X3 = bias_df5

y1 = get_kmeans_classes(player_df5,y)
y2 = get_kmeans_classes(optimal_df5,y)
y3 = get_kmeans_classes(bias_df5,y)

scores5_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores5_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores5_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [51]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores5_1.mean():.3f} - Player Change')
print(f'{scores5_2.mean():.3f} - Optimal Change')
print(f'{scores5_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.583 - Player Change
0.585 - Optimal Change
0.582 - Player Bias



In [52]:
# KMeans Moves Only
X1 = player_df6
X2 = optimal_df6
X3 = bias_df6

y1 = get_kmeans_classes(player_df6,y)
y2 = get_kmeans_classes(optimal_df6,y)
y3 = get_kmeans_classes(bias_df6,y)

scores6_1 = cross_val_score(RandomForestClassifier(), X1, y1)
scores6_2 = cross_val_score(RandomForestClassifier(), X2, y2)
scores6_3 = cross_val_score(RandomForestClassifier(), X3, y3)

In [53]:
print('Baseline Accuracy: 0.020', end='\n\n')

print('Random Forest')
print(f'{scores6_1.mean():.3f} - Player Change')
print(f'{scores6_2.mean():.3f} - Optimal Change')
print(f'{scores6_3.mean():.3f} - Player Bias')
print()

Baseline Accuracy: 0.020

Random Forest
0.602 - Player Change
0.604 - Optimal Change
0.586 - Player Bias

